In [1]:
import numpy as np
import math
import json

In [2]:
def readInputFile(filename):
    #get the data from the input file as a list
    data = []
    with open(filename, "r") as f:
        lines = f.readlines()# easier to type out lines
        #print(lines)
        for i in range(2, len(lines)):
            line = lines[i]
            infoNeeded = line[:line.find(" ")]
            if infoNeeded.lower() != "stable":
                data.append(float(infoNeeded))
            else:
                data.append(0)
    
    #data = np.array(data)
    
    return data

In [3]:
def getHalfLives(data):
    
    halfLives = data[0:3]#gets converted from halflives to lambdas
    

    for i in range(0, len(halfLives)):
        if halfLives[i] != 0:
            halfLives[i] = 0.693 / halfLives[i]
        else:
            halfLives[i] = 0
    lambdas = halflives
    return lambdas

In [4]:
def getInitialConditions(data):
    
    initialConditions = data[3:6]
    timeTotal = data[-2]
    deltaT = data[-1]
    
    return initialConditions, timeTotal, deltaT

In [5]:
def numericallyA(initial, lambdaA, tf, dT):
    
    na = np.zeros(int(tf/dT))
    
    na[0] = initial
    
    for i in range(1, len(na)):
        na[i] = na[i-1] - (dT*na[i-1]*lambdaA)
    
    return na

In [6]:
def numericallyB(initial, lambdaA, lambdaB, tf, dT, na):
    
    nb = np.zeros(int(tf/dT))
    
    nb[0] = initial
    
    for i in range(1, len(nb)):
        nb[i] = nb[i-1] + (dT*na[i-1]*lambdaA) - (dT*nb[i-1]*lambdaB)
    
    return nb

In [7]:
def numericallyC(initial, lambdaB, tf, dT, nb):
    
    nc = np.zeros(int(tf/dT))
    
    nc[0] = initial
    
    for i in range(1, len(nc)):
        nc[i] = nc[i-1] + (dT*nb[i-1]*lambdaB)
    
    return nc

In [8]:
def completeNumericalMethod(inits, lambs, tf, dT):
    
    data = {}
    
    data["$N_A$(t)"] = np.array(numericallyA(inits[0], lambs[0], tf, dT)).tolist()
    
    data["$N_B$(t)"] = np.array(numericallyB(inits[1], lambs[0], lambs[1], tf, dT, data["$N_A$(t)"])).tolist()
    
    data["$N_C$(t)"] = np.array(numericallyC(inits[2], lambs[1], tf, dT, data["$N_B$(t)"])).tolist()
    
    data["$N_{Total}$(t)"] = (np.array(data["$N_A$(t)"]) + np.array(data["$N_B$(t)"]) + np.array(data["$N_C$(t)"])).tolist()
    
    return data

In [9]:
def allNumericalInfo(filename):
    rawInput = readInputFile(filename)
    
    #print(rawInput)
    
    lambs = getHalfLives(rawInput)
    
    inits, tf, dT = getInitialConditions(rawInput)

    return (completeNumericalMethod(inits, lambs, tf, dT))
    

In [10]:
def Nb_with_dif_coarseness(filename):
    rawInput = readInputFile(filename)
    
    #print(rawInput)
    
    lambs = getHalfLives(rawInput)
    
    inits, tf, dT = getInitialConditions(rawInput)
    
    combined = {}

    for i in range(-1,2):
        
        interval = dT / (2**i) #new delta T
        
        data =  completeNumericalMethod(inits, lambs, tf, interval)
    
        title = "$N_B$(t) with $\Delta$t=" + str(interval)
    
        combined[title] = data["$N_B$(t)"]
    
    
    return (combined)

In [11]:
def maximumForNb(filename):
    
    rawInput = readInputFile(filename)
    
    #print(rawInput)
    
    lambs = getHalfLives(rawInput)
    
    inits, tf, dT = getInitialConditions(rawInput)
         
    maximumLocs = []
    deltaTs = []
    
    for i in range(0, 15):
        
        interval = 3*dT / (2**i) #new delta t
        data =  completeNumericalMethod(inits, lambs, tf, interval)
        
        deltaTs.append((1/interval))
        
        maximumLocs.append((np.array(data["$N_B$(t)"]).argmax())*interval)#use location in array to get time of max
    
    
    
    
    return {"deltaTs":deltaTs, "Locations":maximumLocs}

In [12]:
def writeToOutputfile(outputFile, inputFile):
    with open(outputFile, "w") as f:
        
        data1 = allNumericalInfo(inputFile)   
        
        data2 = Nb_with_dif_coarseness(inputFile)
        
        data4 = maximumForNb(inputFile)
        
        data3 = [getHalfLives(readInputFile(inputFile)), getInitialConditions(readInputFile(inputFile))]
        
        json.dump([data1, data2, data4, data3], f)
        
        
        """
        f.write("These are the points for all three elements and the total over the given input time and delta t \n \n \n")
        
        for channel in list(data.keys()):
            f.write("Data for " + channel + "\n")
            f.write("\n")
            f.writelines(str(data[channel]))
            f.write("\n")
            f.write("\n")
        
        f.write("\n")
        f.write("\n")
        
        data = Nb_with_dif_coarseness(inputFile)
        
        for channel in list(data.keys()):
            f.write("Data for " + channel + "\n")
            f.write("\n")
            f.writelines(str(data[channel]))
            f.write("\n")
            f.write("\n")
        """
        
        

In [14]:
writeToOutputfile("Output file.json", "Sample input file.txt")